This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

sVBxpyxPjmRis4jPRogg


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import json
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [34]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json", {'start_date':'2020-05-03','end_date':'2020-05-04','collapse':'hourly','api_key':API_KEY})
print(r.url)
print (r)

https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-05-03&end_date=2020-05-04&collapse=hourly&api_key=sVBxpyxPjmRis4jPRogg
<Response [200]>


In [35]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-05-03', 'end_date': '2020-05-04', 'frequency': 'daily', 'data': [['2020-05-04', 87.65, 88.1, 85.95, 87.65, None, 168947.0, 14745103.15, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [64]:
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json", {'start_date':'2016-12-31','end_date':'2017-12-31','collapse':'daily','api_key':API_KEY})

dic = r.json()
#print(dic['dataset_data']['column_names'])
#print(dic['dataset_data']['data'][0][1])
year_high = 0
year_low = 999999999999999
day_change = 0
two_day_change = 0
holder = None
vols = []
for x in dic['dataset_data']['data']:
    if x[1] != None:
        if x[1] <= year_low:
            year_low = x[1]
        if x[1] >= year_high:
            year_high = x[1]
        if (x[2]-x[3]) >= day_change:
            day_change = (x[2]-x[3])
        if holder != None and abs(x[4] - holder) >= two_day_change:
            two_day_change = abs(x[4] - holder)
        holder = x[4]
        vols.append(x[6])
print('Yearly High:',year_high)
print('Yearly Low:',year_low)
print('Largest Daily Change:',day_change)
print('Largest Two Day Change:',two_day_change)
print('Average Volume:',sum(vols)/len(vols))
vols.sort()
print (len(vols)/2)
if len(vols) % 2 == 0:
    print('Median Volume:',(vols[int(len(vols)/2)] + vols[int((len(vols)/2))+1])/2)
else:
    print('Median Volume:',vols[int(len(vols)/2)])


Yearly High: 53.11
Yearly Low: 34.0
Largest Daily Change: 2.8100000000000023
Largest Two Day Change: 2.559999999999995
Average Volume: 89140.97619047618
126.0


TypeError: list indices must be integers or slices, not float